# HW 1
### Danielle Vaithilingam
**Part 1: Please download Pytorch and one data set from the UCI ML Repository (I recommend Wisconsin Breast Cancer).  Train at least two different architectures (for example, 1 vs. 2 hidden layers) on this data using Pytorch, and show the learning curve (loss function vs. epochs trained).  Describe your architectures and results (two paragraphs are sufficient).**


**Part 2: Please use either ChatGPT (available through OpenAI.com) or GPT4 (available as the New Bing).  Carry out a dialogue on a topic of interest to you (technical or otherwise).  Describe your impressions (2 paragraphs are sufficient).**